System found in **Pymablock: an algorithm and a package for
quasi-degenerate perturbation theory**: arXiv:2404.03728v1

In [1]:
# Objects for defining the Hamiltonian
from sympt import *
# Import sympy
import sympy as sp

# Setup

In [2]:
# ---------------- Defining the symbols ------------------
# Order 0
omega_t = RDSymbol('omega_t', order=0, positive=True, real=True)
omega_r = RDSymbol('omega_r', order=0, positive=True, real=True)
alpha   = RDSymbol('alpha', order=0, positive=True, real=True)
# Order 1
g = RDSymbol('g', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Boson basis transmon: Infinite bosonic Hilbert space
a_t  = BosonOp('a_t')
ad_t = Dagger(a_t)
# Boson basis resonator: Infinite bosonic Hilbert space
a_r  = BosonOp('a_r')
ad_r = Dagger(a_r)

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = omega_t * ad_t * a_t + omega_r * ad_r * a_r + sp.Rational(1,2) * alpha * ad_t * ad_t * a_t * a_t
display(H0)
# Interaction Hamiltonian V
V = -g * (ad_t - a_t) * (ad_r - a_r)
display(V)

alpha*Dagger(a_t)**2*a_t**2/2 + omega_r*Dagger(a_r)*a_r + omega_t*Dagger(a_t)*a_t

-g*(Dagger(a_t) - a_t)*(Dagger(a_r) - a_r)

In [3]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V)

The EffectiveFrame object has been initialized successfully.


# Standard Schrieffer-Wolff Transformation

In [5]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, method="SW")
# Obtaining the result in the dict_operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')

Performing SWT for each order: 100%|████████████████████████████████████████| 2/2 [00:00<00:00,  2.83it/s]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to operator form: 100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 42.45it/s]


In [6]:
# substitution dictionary for transmon ground state
subs_nt0 = {
    a_t : 0,
    ad_t : 0
}

# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [7]:
# displaying 0th order correction
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
# displaying second order correction
grouped_H2 = group_by_operators(H2.subs(subs_nt0).expand())
display_dict({k : v.simplify() for k,v in grouped_H2.items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# ACE

In [9]:
# Deffining the mask
mask = Block(inf=a_r*a_t) +  Block(inf=ad_r*a_t) + Block(inf=a_r**2) + Block(inf=a_t**2*a_r**2) + Block(inf=ad_t*ad_r) + Block(inf=ad_t**2*a_r**2) 
# Calculate the effective model using the Mask routine up to the second order
Eff_frame.solve(max_order=2, method="ACE", mask=mask)
H_eff_Mask = Eff_frame.get_H(return_form='operator')

The perturbative interaction will be added to the full Hamiltonian
Performing Block Diagonalization for each order: 100%|██████████████████████| 2/2 [00:01<00:00,  1.59it/s]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to operator form: 100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 86.09it/s]


In [10]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [19]:
# displaying 0th order corrections
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
# displaying second order corrections
nt = RDSymbol("N_t")
grouped_H2 = group_by_operators(H2.expand())
simplifier = lambda x: sp.Add(*[t.factor() for t in x.subs({ad_t*a_t : nt}).as_ordered_terms()]).collect([1/((nt*alpha-alpha + omega_r + omega_t)**2)
                                                                                                       , 1/((nt*alpha-alpha - omega_r + omega_t)**2),
                                                                                                       1/((nt*alpha - omega_r + omega_t)**2),
                                                                                                       1/((nt*alpha + omega_r + omega_t)**2),
                                                                                                       1/(nt*alpha-alpha+omega_r+omega_t),])
display_dict({k : simplifier(v) for k,v in grouped_H2.items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Full-diagonalization

In [13]:
# Calculate the effective model using the Full Diagonalization routine up to the second order
Eff_frame.solve(max_order=2, method="FD")
H_eff_FD = Eff_frame.get_H(return_form='dict_operator')

The perturbative interaction will be added to the full Hamiltonian
Performing Full Diagonalization for each order: 100%|███████████████████████| 2/2 [00:01<00:00,  1.28it/s]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to dictionary of operator form: 100%|█████████████████████████| 2/2 [00:00<00:00, 11570.49it/s]


In [14]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [15]:
# displaying 0th order corrections
display_dict(H0)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
# displaying 2nd order corrections
simplifier = lambda x: sp.Add(*[t.factor() for t in x.subs({ad_t*a_t : nt}).as_ordered_terms()]).collect([1/((nt*alpha-alpha + omega_r + omega_t)**2)
                                                                                                       , 1/((nt*alpha-alpha - omega_r + omega_t)**2),
                                                                                                       1/((nt*alpha - omega_r + omega_t)**2),
                                                                                                       1/((nt*alpha + omega_r + omega_t)**2),
                                                                                                       1/(nt*alpha-alpha+omega_r+omega_t),])
simplified_dict = {k : simplifier(v) for k,v in H2.items()}
display_dict(simplified_dict)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>